# MIT BIH heart beat classification model for heartbeats


In [ ]:
pip install neurokit.py

In [69]:
import numpy as np
import pandas as pd
import csv
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import tree
%run Functions.ipynb
%run FeaturesFunctions.ipynb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

loaded functions


First we need to create a training file with all of the annotated files in order
File name: 1 csv for feature extraction

In [75]:

AnnotatedFileNames=[*range(100,125)]+[*range(200,235)]
remove=[110,113,120,204,206,211,216,218,224,225,226,227,229]
for i in remove:
    AnnotatedFileNames.remove(i)
Training = pd.DataFrame(columns = ['File','Result', 'Sample#'] )


for i in AnnotatedFileNames:
    temp= pd.DataFrame(columns = Training.columns)

    FileName ="mitbih_database/" + str(i)+'annotations.txt'
    annotations = mlbeats.df_from_txt(FileName)
    temp['Sample#']=annotations['Sample#']
    temp['Result']=annotations['Result']
    temp['File']= i
    
    Training = pd.concat([Training, temp], axis=0)
    

In [76]:
Training.head()
print(len(Training))

110851


Adding neurokit data and aux data to training file.ipynbThen using the nerokit library we can add feature locations to set features in the data, Filename: Adding neurokit data and aux data to training file.ipynb

In [77]:
testFeatures=pd.DataFrame()
for file in AnnotatedFileNames:
    DATA= pd.DataFrame(np.nan, index=range(1), columns=testFeatures.columns)
    testFeatures=pd.concat([testFeatures,DATA])
    testFeatures=pd.concat([testFeatures,Bryson.NerokitExtraction(file)])


extracting File 100


C:\Users\Bryson\anaconda3\lib\site-packages\neurokit2\epochs\eventrelated_utils.py:81: NeuroKitWarning: Input does not have an `ECG_Rate` column. Will skip all rate-related features.
  warn(
C:\Users\Bryson\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:147: NeuroKitWarning: Input does not have an `ECG_Phase_Artrial` or `ECG_Phase_Ventricular` column. Will not indicate whether event onset concurs with cardiac phase.
  warn(
C:\Users\Bryson\anaconda3\lib\site-packages\neurokit2\ecg\ecg_eventrelated.py:170: NeuroKitWarning: Input does not have an `ECG_Quality` column. Quality of the signal is not computed.
  warn(
C:\Users\Bryson\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Bryson\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


extraction complete
extracting File 101
extraction complete
extracting File 102
extraction complete
extracting File 103
extraction complete
extracting File 104
extraction complete
extracting File 105
extraction complete
extracting File 106
extraction complete
extracting File 107
extraction complete
extracting File 108
extraction complete
extracting File 109
extraction complete
extracting File 111
extraction complete
extracting File 112
extraction complete
extracting File 114
extraction complete
extracting File 115
extraction complete
extracting File 116
extraction complete
extracting File 117
extraction complete
extracting File 118
extraction complete
extracting File 119
extraction complete
extracting File 121
extraction complete
extracting File 122
extraction complete
extracting File 123
extraction complete
extracting File 124
extraction complete
extracting File 200
extraction complete
extracting File 201
extraction complete
extracting File 202
extraction complete
extracting File 203


In [80]:
#NewTraining=pd.concat([NewTraining,NewFeatures],axis=1)
Training = Training.reset_index()
testFeatures=testFeatures.reset_index(drop=True)
Training=pd.concat([Training,testFeatures],axis=1)
Training.head()

,level_0,index,File,Result,Sample#,beat_number,ECG_R_Peaks,ECG_Q_Peaks,ECG_S_Peaks,ECG_P_Peaks,...,ECG_Q_Peaks,ECG_S_Peaks,ECG_P_Peaks,ECG_P_Onsets,ECG_P_Offsets,ECG_T_Peaks,ECG_T_Onsets,ECG_T_Offsets,ECG_R_Onsets,ECG_R_Offsets
0,2.0,2.0,100,N,370,2,370.0,347.0,467.0,311.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,3.0,100,N,662,3,662.0,641.0,697.0,606.0,...,55.0,174.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,4.0,100,N,946,4,946.0,926.0,971.0,886.0,...,347.0,467.0,311.0,283.0,344.0,NaN,NaN,NaN,314.0,NaN
3,5.0,5.0,100,N,1231,5,1231.0,1210.0,1323.0,1169.0,...,641.0,697.0,606.0,578.0,638.0,NaN,NaN,NaN,612.0,NaN
4,6.0,6.0,100,N,1515,6,1515.0,1493.0,1609.0,1454.0,...,926.0,971.0,886.0,858.0,923.0,NaN,NaN,NaN,892.0,NaN


In [81]:

Training = Training.drop(["ECG_T_Peaks","ECG_T_Onsets","ECG_T_Offsets","ECG_R_Offsets"],axis='columns')
print(len(Training))
Training.dropna(inplace=True)
print(len(Training))


Interval = pd.DataFrame(columns=["File","Abnormal","Result","R-R Interval","R Height","R_Onset-Rpeak","Q-Q Interval","Q Height","P_Onset-P_Offset","P Height","P-P Interval","P_Onset-Ppeak"])
RRLIST=np.array([])
QQLIST=np.array([])
R_Height=np.array([])
Q_Height=np.array([]) 
P_Height=np.array([])
Ron_RP=np.array([])
Pon_PP=np.array([])
Pon_Poff=np.array([])
PPLIST=np.array([])
Abnormal=np.array([])
Result=np.array([])
FILES=np.array([])
y=2
FileNumber=Training["File"][y]
RAWFile= pd.read_csv("mitbih_database/"+str(FileNumber)+".csv")

for i in Training["index"][:-1]:
    #FileNumber=Training["File"][i]
    #df=pd.DataFrame(columns=Interval.columns)
    #RAWFile= pd.read_csv("mitbih_database/"+str(FileNumber)+".csv")
    
    if (i-y>1):
        y=i
        continue
    if Training["Result"][i]=='N':
        Abnormal=np.append(Abnormal,0)
    else:
        Abnormal=np.append(Abnormal,1)
    if (Training["File"][i] != FileNumber):
        print("Reading File",FileNumber)
        FileNumber=Training["File"][i]
        RAWFile= pd.read_csv("mitbih_database/"+str(FileNumber)+".csv")
    #if (i%100 ==0):
        #print(i,":",time.time()-start)

    RRLIST=np.append(RRLIST,Training["ECG_R_Peaks"][i]-Training["ECG_R_Peaks"][y])
    QQLIST=np.append(QQLIST,Training["ECG_Q_Peaks"][i]-Training["ECG_Q_Peaks"][y])
    PPLIST=np.append(PPLIST,Training["ECG_P_Peaks"][i]-Training["ECG_P_Peaks"][y])
    R_Height=np.append(R_Height,RAWFile[RAWFile.columns[1]][Training["ECG_R_Peaks"][i]])#
    Q_Height=np.append(Q_Height,RAWFile[RAWFile.columns[1]][Training["ECG_Q_Peaks"][i]])#
    P_Height=np.append(P_Height,RAWFile[RAWFile.columns[1]][Training["ECG_P_Peaks"][i]])#
    Ron_RP=np.append(Ron_RP,Training["ECG_R_Peaks"][i]-Training["ECG_R_Onsets"][i])
    Pon_PP=np.append(Pon_PP,Training["ECG_P_Peaks"][i]-Training["ECG_P_Onsets"][i])
    Pon_Poff=np.append(Pon_Poff,Training["ECG_P_Offsets"][i]-Training["ECG_P_Onsets"][i])
    Result=np.append(Result,Training["Result"][i])
    FILES=np.append(FILES,FileNumber)
    
    y=i

Interval["R-R Interval"]=RRLIST
Interval["Q-Q Interval"]=QQLIST
Interval["P-P Interval"]=PPLIST
Interval["R_Onset-Rpeak"]=Ron_RP
Interval["P_Onset-Ppeak"]=Pon_PP
Interval["P_Onset-P_Offset"]=Pon_Poff
Interval["R Height"]=R_Height#
Interval["Q Height"]=Q_Height#
Interval["P Height"]=P_Height#
Interval["File"]=FILES
Interval["Abnormal"]=Abnormal
Interval["Result"]=Result

print("Length of DataFrame",len(Interval))
Interval.head(10)





110851
60924


KeyError: 5.0

In [ ]:
Interval.dropna(inplace=True)
Interval.head()

#creatin
model1 = tree.DecisionTreeClassifier()

inputs = Interval.drop(["Result","Abnormal"],axis='columns')
Target = Interval[["Abnormal"]]



x_train1, x_test1, y_train1, y_test1 = train_test_split(inputs, Target,test_size=0.3)

model1.fit(X=x_train1,y=y_train1)
print(model1.score(x_test1,y_test1),"Regular")
print(roc_auc_score(y_test1, model1.predict(x_test1)),"AUC1\n")



